In [0]:
from pyspark.sql import SparkSession #Importando Pyspark
from pyspark.sql.functions import * #Funções
from pyspark.sql.types import * #Definir tipos específicos
from pyspark.sql.window import Window

In [0]:
df_orders = spark.table("workspace.prata.orders")
df_restaurant = spark.table("workspace.prata.restaurant")
df_teste_ab = spark.table("workspace.prata.teste_ab")
df_users = spark.table("workspace.prata.users")

In [0]:
df_orders.printSchema()

root
 |-- cpf: string (nullable = true)
 |-- id_cliente: string (nullable = true)
 |-- nm_cidade_entrega: string (nullable = true)
 |-- nm_pais_entrega: string (nullable = true)
 |-- nm_bairro_entrega: string (nullable = true)
 |-- nm_estado_entrega: string (nullable = true)
 |-- id_restaurante: string (nullable = true)
 |-- nm_fuso_restaurante: string (nullable = true)
 |-- dt_hr_pedido: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- flg_agendamento: boolean (nullable = true)
 |-- dt_hr_agendamento: string (nullable = true)
 |-- #_vl_total_pedido: double (nullable = true)
 |-- nm_plataforma: string (nullable = true)
 |-- dt_hr_loaded: timestamp (nullable = true)
 |-- dt_hr_pedido_brasilia: timestamp (nullable = true)
 |-- dt_hr_agendamento_brasilia: timestamp (nullable = true)



In [0]:
%sql
CREATE DATABASE ouro;

1 A - Criando indicadores para melhor análise do contexto


Indicador Taxa de Recompra

In [0]:
#KPI: Taxa de Recompra 

#Somando a quantidade de pedidos por cliente e agrupando pelo target e id_cliente
df_recompra = (
    df_orders.join(df_teste_ab, "id_cliente")
    .groupBy("id_cliente", "is_target")
    .agg(count("order_id").alias("qtd_pedidos"))
)


#Se houve recompra seta 1, se não seta 0
df_recompra = df_recompra.withColumn(
    "fez_recompra", when(col("qtd_pedidos") > 1, 1).otherwise(0)
)
#Agrupa pelo target e faz a média para calcular a recompra
df_taxa_recompra = (
    df_recompra.groupBy("is_target")
    .agg(avg("fez_recompra").alias("taxa_recompra"))
)


display(df_taxa_recompra)

#De acordo com o resultado podemos verificar que o grupo de controle obteve 74% de recompra e o grupo de teste obteve 79% de recompra no período.

KPI Média de Pedidos por usuário

In [0]:
# KPI Média de pedidos por usuário
# Média = total de pedidos no grupo / total de usuários no grupo

df_media_pedidos = (
    df_recompra.groupBy("is_target")
    .agg(avg("qtd_pedidos").alias("media_pedidos_por_usuario"))
)


display(df_media_pedidos)
#Observamos que a quantidade de pedidos por usuário no grupo de teste é maior que no grupo de controle. sendo 4.23 pedidos para o controle e 4.79 para pedidos de teste.


KPI TIcket Médio por usuário

In [0]:
# TICKET MÉDIO POR USUÁRIO
# ticket médio = valor total dos pedidos do grupo / n de usuário do grupo

#Verificando o valor total gasto por cliente
df_gasto_cliente = (
    df_orders.join(df_teste_ab, "id_cliente")
    .groupBy("id_cliente", "is_target")
    .agg(sum("#_vl_total_pedido").alias("vl_total_cliente"))
)

#Calculando o ticket médio por cliente
df_ticket_medio_usuario = (
    df_gasto_cliente.groupBy("is_target")
    .agg(avg("vl_total_cliente").alias("ticket_medio_usuario"))
)

display(df_ticket_medio_usuario)

#Nesse indicador, verificamos mais uma vez que os usuário do grupo de experimento, gasta mais do que os usuários do grupo de controle no périodo, outro possível impacto da campanha de cupons.

Ticket Médio por Pedido

In [0]:
# Ticket médio por pedido
# ticket médio por pedido = soma dos valores dos pedidos / número total de pedidos
#Cada pedido, em média, ficou em quanto?

df_pedidos = df_orders.join(df_teste_ab, "id_cliente")

df_ticket_medio_pedido = (
    df_pedidos
    .groupBy("is_target")
    .agg(avg("#_vl_total_pedido").alias("ticket_medio_pedido"))
)

display(df_ticket_medio_pedido)

#Aqui não houve diferença, isso indica que o cupom não aumentou o valor médio por pedido, mas sim o volume de pedidos.


KPI Retenção D+7 e D+14

In [0]:
# KPI: Retenção D+7 / D+14
# % de usuários que fizeram pelo menos 1 pedido adicional dentro de 7 ou 14 dias após o primeiro pedido.


# Data do primeiro pedido de cada cliente
df_first_order = (
    df_orders
    .groupBy("id_cliente")
    .agg(min("dt_hr_pedido_brasilia").alias("dt_primeiro_pedido"))
)

#Juntanto com pedidos e calculando o dia desde o primeiro pedido
df_pedidos_com_dia = (
    df_orders.join(df_first_order, "id_cliente")
    .withColumn("dias_desde_primeiro", datediff("dt_hr_pedido_brasilia", "dt_primeiro_pedido"))
)




# D+7
#Filtrando pedidos entre 1 e 7 dias
df_retenidos_d7 = (
    df_pedidos_com_dia
    .filter((col("dias_desde_primeiro") > 0) & (col("dias_desde_primeiro") <= 7))
    .select("id_cliente")
    .distinct()
    .withColumn("retenido_d7", lit(1))
)

# D+14
# Filtrando pedidos entre 1 e 14 dias
df_retenidos_d14 = (
    df_pedidos_com_dia
    .filter((col("dias_desde_primeiro") > 0) & (col("dias_desde_primeiro") <= 14))
    .select("id_cliente")
    .distinct()
    .withColumn("retenido_d14", lit(1))
)

#Juntando pedidos com retenção de 7 e 14
df_recompra_retencao = (
    df_orders.select("id_cliente").distinct()
    .join(df_retenidos_d7, "id_cliente", "left")
    .join(df_retenidos_d14, "id_cliente", "left")
    .fillna(0)
)


df_recompra_retencao = df_recompra_retencao.join(df_teste_ab, "id_cliente")

df_retencao = (
    df_recompra_retencao.groupBy("is_target")
    .agg(
        avg("retenido_d7").alias("taxa_retencao_d7"),
        avg("retenido_d14").alias("taxa_retencao_d14")
    )
)

display(df_retencao)

# Grupo de teste teve uma taxa de retenção maior tanto em 7 quanto em 14 dias, mostrando que houve uma maior recorrência dentro de 2 semanas.

Ticket Médio entre Pedidos

In [0]:
# Tempo médio entre pedidos
# tempo médio = diferença entre pedidos consecutivos / número de intervalos
# Quanto menor o tmepo médio, maior o engajamento

#Join orders e grupo de teste
df_pedidos = df_orders.join(df_teste_ab, "id_cliente")

#Windows funciont, agrupando os dados por cliente
janela = Window.partitionBy("id_cliente").orderBy("dt_hr_pedido_brasilia")

#Adicionando colunsa pedido anterior e dias entre pedidos
df_diff_pedidos = (
    df_pedidos
    .withColumn("pedido_anterior", lag("dt_hr_pedido_brasilia").over(janela))
    .withColumn("dias_entre_pedidos", datediff("dt_hr_pedido_brasilia", "pedido_anterior"))
)

#Adicionando coluna de tempo medio de clientes
df_tempo_medio_cliente = (
    df_diff_pedidos
    .groupBy("id_cliente", "is_target")
    .agg(avg("dias_entre_pedidos").alias("tempo_medio_cliente"))
    .filter(col("tempo_medio_cliente").isNotNull())  # só quem tem mais de 1 pedido
)

#Pegando a média do tempo e agrupando por grupo.
df_tempo_medio_grupo = (
    df_tempo_medio_cliente
    .groupBy("is_target")
    .agg(avg("tempo_medio_cliente").alias("tempo_medio_entre_pedidos"))
)



display(df_tempo_medio_grupo)

# 16.7 dias control
# 14.9 dias target
# Grupo de teste teve um tempo médio entre pedidos menor, nesse caso o cupom pode ter acelerado o ciclo de recompra fazendo o cliente do grupo de teste pedir mais cedo.


In [0]:
#Juntando todos os Kpis e persistindo em tabela.
df_kpis_avaliacao = df_taxa_recompra \
    .join(df_media_pedidos, on="is_target", how="inner") \
    .join(df_ticket_medio_usuario, on="is_target", how="inner")\
    .join(df_ticket_medio_pedido, on="is_target", how="inner")\
    .join(df_retencao, on="is_target", how="inner")\
    .join(df_tempo_medio_grupo, on="is_target", how="inner")

df_kpis_avaliacao.write.format("delta").mode("overwrite").saveAsTable("Ouro.tbl_kpi_avaliacao")

In [0]:
kpis_visu = spark.table("workspace.ouro.tbl_kpi_avaliacao")

kpis_visu.display()

1 B - Análise de Viabilidade Financeira

In [0]:
from pyspark.sql.functions import sum as _sum

Calculando indicadores financeiros 
- Receita incremental  
- Custo Total  
- Lucro Incremental  
- ROI

In [0]:

#Verificando a receita total por grupo
df_receita = (
    df_orders.join(df_teste_ab, "id_cliente")
    .groupBy("is_target")
    .agg(_sum("#_vl_total_pedido").alias("receita_total"))
)

#Quantidade total de pedidos por grupo
df_pedidos = (
    df_orders.join(df_teste_ab, "id_cliente")
    .groupBy("is_target")
    .agg(count("order_id").alias("qtd_pedidos"))
)

#Definindo o custo médio de cada cupom a 10 reais.
valor_cupom = 10.0

#Adicionando o campo custo_cupom, que trás o custo total de cupons por grupo
df_financeiro = (
    df_receita.join(df_pedidos, "is_target")
    .withColumn("custo_cupom", when(col("is_target") == "target", col("qtd_pedidos") * valor_cupom).otherwise(0.0))
)




# Pegando o primeiro registro de cada df
controle = df_financeiro.filter(col("is_target") == "control").collect()[0]
tratamento = df_financeiro.filter(col("is_target") == "target").collect()[0]

# Receita incremental
receita_inc = tratamento["receita_total"] - controle["receita_total"]

# Custo total
custo_total = tratamento["custo_cupom"]

# Lucro incremental
lucro_incremental = receita_inc - custo_total

# ROI = Retorno sobre o investimento 
roi = lucro_incremental / custo_total if custo_total != 0 else None


print(f"Receita Incremental: {receita_inc}")
print(f"Custo total: {custo_total}")
print(f"Lucro Incremental: {lucro_incremental}")
print(f"ROI: {roi}") 

#display(receita_inc) #28 935 696.96
#display(custo_total) # 21 367 450
#display(lucro_incremental) #7 568 246.96
display(roi) #35%

#De acordo com os resultados dos indicadores, a campanha de cupons foi financeiramente viável, pois o VALOR DE RECEITA ADICIONAL no grupo de tratamento superou os custos de descontos e o LUCRO foi de 7,57 milhões. O roi foi de 35% e indica que para cada 1 R$ investido em cupom, o retorno foi de 1,35.




Parte 2 - Segmentação e reanalise dos resultados para o novo contexto segmentado.

Segmentando por Baixa, Média e Alta Frequência  
Critérios:  
Baixa -> 1 pedido  
Média -> 2-3 pedidos  
Alta -> 4 ou mais pedidos    

In [0]:
#Contando a quantidade de pedidos por cliente
df_freq = df_orders.groupBy("id_cliente").agg(count("*").alias("num_pedidos"))

#Verificando a frequencia de compra por cliente
df_freq_segmentado = df_freq.withColumn(
    "segmento_frequencia",
    when(df_freq.num_pedidos == 1, "baixa") \
    .when((df_freq.num_pedidos >= 2) & (df_freq.num_pedidos <= 3), "media") \
    .otherwise("alta")
)

df_freq_segmentado2 = df_freq_segmentado.join(df_teste_ab, on="id_cliente", how="left")

df_freq_segmentado2 = df_freq_segmentado2.groupBy("segmento_frequencia", "is_target") \
    .agg(
        countDistinct("id_cliente").alias("total_clientes"),
        sum("num_pedidos").alias("total_pedidos")
    ) \
    .filter(col("is_target").isNotNull())\
    .orderBy("segmento_frequencia", "is_target")


display(df_freq_segmentado2)

# Vimos que clientes a quantidade de clientes com alta e média frequência é maior no grupo de testes do que no grupo de controle

# Para clientes de baixa frequência, o número é praticamente o mesmo.


Segmentado por Estado

In [0]:
# Agrupando quantidade de pedidos por cliente
df_freq = df_orders.groupBy("id_cliente").agg(count("*").alias("num_pedidos"))

#Pegando o estado de entrega do cliente (do último pedido, por exemplo)
# Assumindo que você pode usar o primeiro estado encontrado
df_estado = df_orders.select("id_cliente", "nm_estado_entrega").dropna(subset=["nm_estado_entrega"]).dropDuplicates(["id_cliente"])

#Junta com a tabela de teste_ab e quantidade de pedidos
df_estado_segmentado = df_freq \
    .join(df_estado, on="id_cliente", how="left") \
    .join(df_teste_ab, on="id_cliente", how="left")

# 4. Agrupa por estado e grupo de teste
df_estado_agrupado = df_estado_segmentado.groupBy("nm_estado_entrega", "is_target") \
    .agg(
        countDistinct("id_cliente").alias("total_clientes"),
        sum("num_pedidos").alias("total_pedidos")
    ) \
    .filter(col("is_target").isNotNull()) \
    .orderBy("nm_estado_entrega", "is_target")


df_estado_agrupado.display()


#Observamos que em todos os estados, a quantidade de pedidos cresceu no grupo de teste, em relação ao grupo de controle.


Refazendo o ROI por tipo de perfil de cliente

In [0]:
# Juntando os dados
df_base = df_orders.join(df_teste_ab, "id_cliente", "inner") \
                   .join(df_freq_segmentado.select("id_cliente", "segmento_frequencia"), "id_cliente", "left")

# Receita por segmento e grupo
df_receita = df_base.groupBy("segmento_frequencia", "is_target") \
    .agg(_sum("#_vl_total_pedido").alias("receita_total"))

# Pedidos por segmento e grupo
df_pedidos = df_base.groupBy("segmento_frequencia", "is_target") \
    .agg(count("order_id").alias("qtd_pedidos"))

# Unindo receita e pedidos
df_financeiro = df_receita.join(df_pedidos, ["segmento_frequencia", "is_target"])

# Custo do cupom apenas para o grupo "target"
df_financeiro = df_financeiro.withColumn(
    "custo_cupom",
    when(col("is_target") == "target", col("qtd_pedidos") * valor_cupom).otherwise(0.0)
)

# Pivotando para calcular Receita Incremental e ROI por segmento
df_final = df_financeiro.groupBy("segmento_frequencia").pivot("is_target", ["control", "target"]) \
    .agg(
        _sum("receita_total").alias("receita_total"),
        _sum("custo_cupom").alias("custo_total")
    ) \
    .withColumn("receita_incremental", col("target_receita_total") - col("control_receita_total")) \
    .withColumn("lucro_incremental", col("receita_incremental") - col("target_custo_total")) \
    .withColumn("roi", (col("lucro_incremental") / col("target_custo_total")) * 100)


df_final.select(
    "segmento_frequencia",
    "receita_incremental",
    "target_custo_total",
    "lucro_incremental",
    "roi"
).display()


#Observamos que clientes que tem frequencia baixa e média, não dão bons resultados financeiros, porém clientes que tem alta frequencia, tem bons resultados, nesse caso o resultado foi de 56% de retorno sobre o investimento.

Refazendo o ROI por Estado

In [0]:
#Reavaliação por Estado
# 1. Juntando os dados necessários
df_base_estado = df_orders.join(df_teste_ab, "id_cliente", "inner") \
                          .select("id_cliente", "order_id", "nm_estado_entrega", "#_vl_total_pedido", "is_target")

# 2. Receita total por estado e grupo
df_receita_estado = df_base_estado.groupBy("nm_estado_entrega", "is_target") \
    .agg(_sum("#_vl_total_pedido").alias("receita_total"))

# 3. Quantidade de pedidos por estado e grupo
df_pedidos_estado = df_base_estado.groupBy("nm_estado_entrega", "is_target") \
    .agg(count("order_id").alias("qtd_pedidos"))

# 4. Unindo as métricas
df_financeiro_estado = df_receita_estado.join(df_pedidos_estado, ["nm_estado_entrega", "is_target"])

# 5. Custo do cupom apenas para grupo "target"
df_financeiro_estado = df_financeiro_estado.withColumn(
    "custo_cupom",
    when(col("is_target") == "target", col("qtd_pedidos") * valor_cupom).otherwise(0.0)
)

# 6. Pivotando os dados para comparação
df_final_estado = df_financeiro_estado.groupBy("nm_estado_entrega").pivot("is_target", ["control", "target"]) \
    .agg(
        _sum("receita_total").alias("receita_total"),
        _sum("custo_cupom").alias("custo_total")
    ) \
    .withColumn("receita_incremental", col("target_receita_total") - col("control_receita_total")) \
    .withColumn("lucro_incremental", col("receita_incremental") - col("target_custo_total")) \
    .withColumn("roi", (col("lucro_incremental") / col("target_custo_total")) * 100)

# 7. Exibindo o resultado
df_final_estado.select(
    "nm_estado_entrega",
    "receita_incremental",
    "target_custo_total",
    "lucro_incremental",
    "roi"
).orderBy("roi").display()


#Verificamos que alguns estados como SC, PB, MA os resultados não foram bons
#Já estados como BA, SP e RJ tiverem excelentes resultados